In [10]:
#Import pandas
import pandas as pd

In [11]:
#import yelp csv to clean up
file_path='../Resources/yelpdata.csv'
yelp_df=pd.read_csv(file_path,index_col=0)
yelp_df.head()

,locationID,latitude,longitude,category,results_total,avg_rating,avg_review_count
0,0,38.046407,-84.497039,beaches,2,4.75,3.500
1,0,38.046407,-84.497039,horsebackriding,16,4.50,13.375
2,0,38.046407,-84.497039,hiking,8,4.50,8.500
3,0,38.046407,-84.497039,paddleboarding,2,4.75,16.500
4,0,38.046407,-84.497039,ziplining,20,5.00,7.900


In [12]:
#brought over note from aggregate workbook, standardizing categories across datasets
# Clean final DF Activities to be one of the following list: 
#       'beaches', 'horsebackriding', 'hiking', 'paddleboarding',
#        'ziplining', 'spas', 'massage', 'tours', 'hanggliding',
#        'hot_air_balloons', 'parasailing', 'sailing', 'snorkeling',
#        'hotsprings'

In [14]:
#Should effectively drop all rows where the category column does not contain one of the above values. 
yelp_df=yelp_df[yelp_df['category'].str.contains("beaches|hanggliding|horsebackriding|hiking|hot_air_balloons|paddleboarding|parasailing|sailing|snorkeling|ziplining|Spas|hotsprings|massage|tours")== True]

In [15]:
['Horsebackriding', 'Temples', 'Paddleboarding', 'nan', 'Whale Watching', 'Snorkeling', 'Parasailing', 'Tours', 
 'Wine Tours', 'Hanggliding', 'Beaches', 'Hot_Air_Balloons', 'Spas', 'Hot Springs', 'Sailing', 'Hiking', 'Massage', 'Ziplining', 'Hotsprings', 'Hot Air Balloons', 'Hang Gliding']



['Horsebackriding',
 'Temples',
 'Paddleboarding',
 'nan',
 'Whale Watching',
 'Snorkeling',
 'Parasailing',
 'Tours',
 'Wine Tours',
 'Hanggliding',
 'Beaches',
 'Hot_Air_Balloons',
 'Spas',
 'Hot Springs',
 'Sailing',
 'Hiking',
 'Massage',
 'Ziplining',
 'Hotsprings',
 'Hot Air Balloons',
 'Hang Gliding']

In [16]:
#create a 'top rated' column based on the average rating and number of reviews 
yelp_df['top_rated'] = yelp_df['avg_rating'] * yelp_df['avg_review_count'] 
yelp_df.head()

,locationID,latitude,longitude,category,results_total,avg_rating,avg_review_count,top_rated
0,0,38.046407,-84.497039,beaches,2,4.75,3.500,16.6250
1,0,38.046407,-84.497039,horsebackriding,16,4.50,13.375,60.1875
2,0,38.046407,-84.497039,hiking,8,4.50,8.500,38.2500
3,0,38.046407,-84.497039,paddleboarding,2,4.75,16.500,78.3750
4,0,38.046407,-84.497039,ziplining,20,5.00,7.900,39.5000


In [17]:
#group by [lat,lon] pair and sort top rated value to descending
yelp_df = yelp_df.groupby(['locationID']).apply(lambda x: x.sort_values(by='top_rated', ascending=False))
yelp_df.head()

locationID   latitude  longitude         category  \
locationID                                                        
0          7           0  38.046407 -84.497039            tours   
           3           0  38.046407 -84.497039   paddleboarding   
           1           0  38.046407 -84.497039  horsebackriding   
           4           0  38.046407 -84.497039        ziplining   
           2           0  38.046407 -84.497039           hiking   

              results_total  avg_rating  avg_review_count  top_rated  
locationID                                                            
0          7             20       4.875            35.000   170.6250  
           3              2       4.750            16.500    78.3750  
           1             16       4.500            13.375    60.1875  
           4             20       5.000             7.900    39.5000  
           2              8       4.500             8.500    38.2500

In [ ]:
#get rid of that index.
yelp_df=yelp_df.reset_index(drop=True)
yelp_df.head()

In [ ]:
#group by again and drop all values below the top five for each [lat,lon] pair
yelp_df=yelp_df.groupby(['latitude','longitude']).head(5)
yelp_df.head(15)

In [ ]:
# group again to convert the category column into a list and collapse the [lat,lon] pairs into a single row
grouped_df = yelp_df.groupby(['latitude', 'longitude']).agg({'category': lambda x: list(x)})
grouped_df.head()

In [ ]:
#loop through the locations and expand the category list into separate columns
for i in range(5):
    grouped_df[f'act{i+1}'] = grouped_df['category'].apply(lambda x: x[i] if len(x) > i else None)
grouped_df.head()

In [ ]:
#drop the original category column
grouped_df=grouped_df.drop(['category'],axis=1)
grouped_df.head()

In [ ]:
yelp_df=yelp_df.reset_index(drop=True)
yelp_df.head()

In [ ]:
### export it  as final_activities.csv
grouped_df.to_csv('../Resources/final_activities.csv',index=True)